# LSEG COMPANY TEARSHEET

In [1]:
RIC = input()
# print(RIC)

 IBM.N


In [2]:
# Import module and set up with App Key
import lseg.data as ld
APP_KEY = '<App Key>'
ld.open_session(app_key = APP_KEY)
import pandas as pd 
import numpy as np

In [3]:
# Request Header Information for RIC 
df = ld.get_data(RIC, "TR.CommonName(),TR.HeadquartersCountry,TR.TRBCIndustryGroup,TR.GICSSector,TR.CompanyMarketCap.Currency")
#Request Exchange for Primary Quote RIC
df_2 = ld.get_data(RIC,"TR.PrimaryQuote")
df_3 = ld.get_data(df_2.at[0,'Primary Quote RIC'],"TR.ExchangeName")  
df['Exchange Name'] = [df_3.at[0,'Exchange Name']]
df

,Instrument,Company Common Name,Country of Headquarters,TRBC Industry Group Name,GICS Sector Name,Currency,Exchange Name
0,IBM.N,International Business Machines Corp,United States of America,Software & IT Services,Information Technology,USD,"NEW YORK STOCK EXCHANGE, INC."


## General Information

In [4]:
#As single Dataframe - example only
#dfGenSingle = ld.get_data(RIC, 
#        [{'TR.PriceClose;TR.Price52WeekHigh;TR.PricePctChg52WkHigh;TR.Price52WeekLow;TR.PricePctChg52WkLow;TR.PriceTargetMedian': {'Curn':df.at[0,'Currency']}},
#            {'TR.BetaFiveYear;TR.DividendYield;TR.SharesOutstanding;TR.FreeFloat;TR.FreeFloatPct();TR.CompanyMarketCap;TR.EV': {'Scale':6,'Curn':df.at[0,'Currency']}}])

#As two Dataframes - more convenient
dfGen = ld.get_data(RIC, 'TR.PriceClose;TR.Price52WeekHigh;TR.PricePctChg52WkHigh;TR.Price52WeekLow;TR.PricePctChg52WkLow;TR.PriceTargetMedian',{'Curn':df.at[0,'Currency']})
dfGen2 = ld.get_data( RIC, 'TR.BetaFiveYear;TR.DividendYield;TR.SharesOutstanding;TR.FreeFloat;TR.FreeFloatPct();TR.CompanyMarketCap;TR.EV',{'Scale':6,'Curn':df.at[0,'Currency']})

#calc implied profit
dfGen['Implied Profit'] = (dfGen.at[0,'Price Target - Median']/dfGen.at[0,'Price Close'] - 1)*100

# results as simple lists 
listGenNames = dfGen.columns.tolist()
listGen = dfGen.iloc[0].tolist()
listGen2Names = dfGen2.columns.tolist()
listGen2 = dfGen2.iloc[0].tolist()

#remove RIC
listGenNames.pop(0)
listGen.pop(0)
listGen2Names.pop(0)
listGen2.pop(0)

dataDictGen= {'1': listGenNames, '2': listGen , '3': listGen2Names, '4': listGen2} 
dfGenX = pd.DataFrame(dataDictGen)  

dfGenX

,1,2,3,4
0,Price Close,236.220000,Beta 5 Year,0.628409
1,Price 52 Week High,266.300000,Dividend yield,2.797203
2,52-week High Price PCT Change,-11.295531,Outstanding Shares,927.264332
3,Price 52 Week Low,162.650000,Free Float,926.039119
4,52-week Low Price PCT Change,45.232093,Free Float (Percent),99.867900
5,Price Target - Median,262.495000,Company Market Cap,219038.380505
6,Implied Profit,11.123106,Enterprise Value (Daily Time Series),259506.380505


## Consensus Recommendation

In [5]:
df6 = ld.get_data(RIC, 'TR.RecLabel();TR.NumOfRecommendations();TR.NumEstRevisingUp;TR.NumEstRevisingDown',{'WP':'30d','EstimateMeasure':'EPS'})
df6

,Instrument,Recommendation - Mean Label,Recommendation - Number Of Total,Number of Analysts Revising Up,Number of Analysts Revising Down
0,IBM.N,BUY,1970-01-01 00:00:00.000000023,3,5


## Estimates

In [8]:
#Retrieve Estimates into Data Frames per Period

#FY0
dfFY0 = ld.get_data(RIC, 'TR.RevenueActValue;TR.EBITDAActValue;TR.EPSActValue',
                       {'Period':'FY0', 'Scale':'6', 'Curn':df.at[0,'Currency']})
#FY1
dfFY1 = ld.get_data(RIC, 
'TR.RevenueMeanEstimate;TR.RevenueHigh;TR.RevenueLow;TR.EBITDAMean;TR.EBITDAHigh;TR.EBITDALow;TR.EPSMeanEstimate;TR.EPSHigh;TR.EPSLow',
                       {'Period':'FY1', 'scale':'6', 'curn':df.at[0,'Currency']})
dfFY1_2 = ld.get_data(RIC, 
                           'TR.RevenueMeanEstimate; TR.EBITDAMean; TR.EPSMeanEstimate',
    {'Period':'FY1','RollPeriods':'false','Scale': 6,'Curn':df.at[0,'Currency'],'SDate':'-1AM'})
#FY2
dfFY2 = ld.get_data(RIC, 
'TR.RevenueMeanEstimate;TR.RevenueHigh;TR.RevenueLow;TR.EBITDAMean;TR.EBITDAHigh;TR.EBITDALow;TR.EPSMeanEstimate;TR.EPSHigh;TR.EPSLow',
                       {'Period':'FY2', 'scale':'6', 'curn':df.at[0,'Currency']})
dfFY2_2 = ld.get_data(RIC, 'TR.RevenueMeanEstimate;TR.EBITDAMean;TR.EPSMeanEstimate',
    {'Period':'FY2','RollPeriods':'false','Scale': 6,'Curn':df.at[0,'Currency'],'SDate':'-1AM'})

### DEBUG display(dfFY0,dfFY1,dfFY1_2, dfFY2, dfFY2_2)

In [9]:
#Create a dictionary out of results, and structure a single multi-row Data Frame

# initialise results per column 
data = {'Sales (MIL) Mean EST':
        [dfFY0.at[0,'Revenue - Actual'], dfFY1.at[0,'Revenue - Mean Estimate'],'',dfFY2.at[0,'Revenue - Mean Estimate'],''],
       'High Est':
       ['',dfFY1.at[0,'Revenue - High'],(dfFY1.at[0,'Revenue - High']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2.at[0,'Revenue - High'],(dfFY2.at[0,'Revenue - High']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'Low Est':
       ['',dfFY1.at[0,'Revenue - Low'],(dfFY1.at[0,'Revenue - Low']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2.at[0,'Revenue - Low'],(dfFY2.at[0,'Revenue - Low']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'Est. 30 30 Days Ago':
       ['',dfFY1_2.at[0,'Revenue - Mean Estimate'],(dfFY1_2.at[0,'Revenue - Mean Estimate']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2_2.at[0,'Revenue - Mean Estimate'],(dfFY2_2.at[0,'Revenue - Mean Estimate']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'EBITDA (MIL) Mean Est.':
        [dfFY0.at[0,'EBITDA - Actual'],dfFY1.at[0,'EBITDA - Mean'],'',dfFY2.at[0,'EBITDA - Mean'],''],
        'High Est.':
        ['',dfFY1.at[0,'EBITDA - High'],(dfFY1.at[0,'EBITDA - High']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
         dfFY2.at[0,'EBITDA - High'],(dfFY2.at[0,'EBITDA - High']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'Low Est.':
        ['',dfFY1.at[0,'EBITDA - Low'],(dfFY1.at[0,'EBITDA - Low']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
         dfFY2.at[0,'EBITDA - Low'],(dfFY2.at[0,'EBITDA - Low']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'Est. 30 30 Days Ago.':
       ['',dfFY1_2.at[0,'EBITDA - Mean'],(dfFY1_2.at[0,'EBITDA - Mean']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
        dfFY2_2.at[0,'EBITDA - Mean'],(dfFY2_2.at[0,'EBITDA - Mean']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'EPS Mean Est.':
        [dfFY0.at[0,'Earnings Per Share - Actual'], dfFY1.at[0,'Earnings Per Share - Mean Estimate'],'',
         dfFY2.at[0,'Earnings Per Share - Mean Estimate'],''],
        'High Est..':
         ['',dfFY1.at[0,'Earnings Per Share - High'],(dfFY1.at[0,'Earnings Per Share - High']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2.at[0,'Earnings Per Share - High'],(dfFY2.at[0,'Earnings Per Share - High']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100],
        'Low Est..':
         ['',dfFY1.at[0,'Earnings Per Share - Low'],(dfFY1.at[0,'Earnings Per Share - Low']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2.at[0,'Earnings Per Share - Low'],(dfFY2.at[0,'Earnings Per Share - Low']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100],
        'Est. 30 30 Days Ago..':
        ['',dfFY1_2.at[0,'Earnings Per Share - Mean Estimate'],(dfFY1_2.at[0,'Earnings Per Share - Mean Estimate']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2_2.at[0,'Earnings Per Share - Mean Estimate'],(dfFY2_2.at[0,'Earnings Per Share - Mean Estimate']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100]
       }  
  
# Create Pandas DataFrame. 
dfMultiRow = pd.DataFrame(data, index =['FY0 (Actual)','FY1 (Current Year)', '% above/below mean','FY2 (Next Year)','% above/below mean' ]) 
                                             
dfMultiRow                     

,Sales (MIL) Mean EST,High Est,Low Est,Est. 30 30 Days Ago,EBITDA (MIL) Mean Est.,High Est.,Low Est.,Est. 30 30 Days Ago.,EPS Mean Est.,High Est..,Low Est..,Est. 30 30 Days Ago..
FY0 (Actual),62753,,,,16400,,,,10.33,,,
FY1 (Current Year),65076.02167,66377,63800,64974.57225,17039.26313,18064,15860,17067.61713,10.76595,11.15,10.079,10.71837
% above/below mean,,1.999167,-1.960817,-0.155894,,6.013974,-6.920858,0.166404,,3.567265,-6.380765,-0.441949
FY2 (Next Year),68075.7141,70080,66440.044,68099.71453,17845.78863,19303.743,16800,17894.22313,11.42021,12.29,10.526,11.41222
% above/below mean,,2.944201,-2.402722,0.035255,,8.169739,-5.860142,0.271406,,7.616235,-7.830066,-0.069964


# Valuation Ratios

In [10]:
# Retrieve Valuation Ratios into DataFrames per period

dfLTM = ld.get_data(RIC,'TR.PE;TR.PriceToSalesPerShare;TR.EVToSales;TR.EVToEBITDA',{'Sdate':'0D'})
dfFY1 = ld.get_data(RIC, 'TR.PToEPSMeanEst;TR.PToREVMeanEst',{'Period':'FY1'})
dfFY1_2 = ld.get_data(RIC,'TR.EV;TR.RevenueMeanEstimate,TR.EBITDAMean',{'Period':'FY1','Curn':df.at[0,'Currency']})
dfFY2 = ld.get_data(RIC, 'TR.PToEPSMeanEst;TR.PToREVMeanEst',{'Period':'FY2'})
dfFY2_2 = ld.get_data(RIC,'TR.EV;TR.RevenueMeanEstimate,TR.EBITDAMean',{'Period':'FY2','Curn':df.at[0,'Currency']})
###DEBUG display(dfLTM,dfFY1,dfFY1_2,dfFY2, dfFY2_2)

C:\Python312\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Python312\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast


In [11]:
#Create a dictionary out of results, and structure a single multi-row Data Frame

# Initialise results per column 
dataDict = {'P/E':
        [dfLTM.at[0,'P/E (Daily Time Series Ratio)'], dfFY1.at[0,'Price / EPS (Mean Estimate)'], dfFY2.at[0,'Price / EPS (Mean Estimate)']],
            'P/SALES':
        [dfLTM.at[0,'Price To Sales Per Share (Daily Time Series Ratio)'], dfFY1.at[0,'Price / Revenue (Mean Estimate)'], dfFY2.at[0,'Price / Revenue (Mean Estimate)']],
            'EV/REVS':
                [dfLTM.at[0,'Enterprise Value To Sales (Daily Time Series Ratio)'], 
                 dfFY1_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY1_2.at[0,'Revenue - Mean Estimate'], 
                 dfFY2_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY2_2.at[0,'Revenue - Mean Estimate']],
            
            'EV/EBITDA':
                [dfLTM.at[0,'Enterprise Value To EBITDA (Daily Time Series Ratio)'], 
                 dfFY1_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY1_2.at[0,'EBITDA - Mean'], 
                 dfFY2_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY2_2.at[0,'EBITDA - Mean']]
}

#Create Pandas DataFrame. 
dfMultiRow = pd.DataFrame(dataDict, index =['LTM (Last Twelve Months)','FY1 (Current Year)', 'FY2 (Next Year)']) 
dfMultiRow        

,P/E,P/SALES,EV/REVS,EV/EBITDA
LTM (Last Twelve Months),34.255634,3.490429,4.135296,17.837942
FY1 (Current Year),21.941399,3.365885,3.987742,15.229906
FY2 (Next Year),20.684383,3.217570,3.812026,14.541603


## FY0 Per Share Metrics 

In [12]:
dfFYOShare = ld.get_data(RIC ,['TR.TotCashAndShortTermInvestmentsperShare',
'TR.WorkingCapitalperShare','BookValuePerShare','TR.RevenuePerShare','TR.CashFlowtoShrdil',
'TR.FreeOperatingCashFlowperShareAvgDilutedSharesOut','TR.DPSCommonStock'], {'Period':'FY0','curn':df.at[0,'Currency']})
dfFYOShare_2 = ld.get_data(RIC ,['TR.TotalDebt','TR.TtlCmnSharesOut'],{'Period':'FY0','curn':df.at[0,'Currency']})
# additionally, calculate total debt
dfFYOShare['Total Debt'] = [dfFYOShare_2.at[0,'Total Debt']/dfFYOShare_2.at[0,'Total Common Shares Outstanding']]
dfFYOShare

C:\Python312\Lib\site-packages\lseg\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


,Instrument,Total Cash & ST Invest/Share,Wk Cap/Shr,Revenue Per Share,Cash Flow/Shr (dil.),FOCF/Shr (dil.),DPS - Common Stock Primary Issue,BOOKVALUEPERSHARE,Total Debt
0,IBM.N,15.752085,1.446631,66.960731,11.883761,5.990431,6.67,<NA>,59.3475


## FY0 Employee Data

In [13]:
dfEmp = ld.get_data(RIC,'TR.Employees(Period=FY0),TR.RevenueperEmployeescalYrEnd(Period=FY0)')
dfEmp_2= ld.get_data(RIC,'TR.Employees(Period=FY-1)')
dfEmp_3 = ld.get_data(RIC,'TR.RevenueperEmployeescalYrEnd(Period=FY0,sdate=0D,Edate=-4FY,Frq=FY)')
dfEmp['Empt Grth'] = (dfEmp_2.at[0,'Full-Time Employees']-dfEmp.at[0,'Full-Time Employees'])/dfEmp_2.at[0,'Full-Time Employees']
dfEmp['5 yr Avg'] = dfEmp_3["Revenue/Employee, FY"].mean()
dfEmp

,Instrument,Full-Time Employees,"Revenue/Employee, FY",Empt Grth,5 yr Avg
0,IBM.N,293400,209630.866878,0.038978,185091.26929


## Abbreviated Financials

### Income Statement (MIL)

In [14]:
dfFin = ld.get_data(RIC, 'TR.TotalRevenue;TR.GrossProfit;TR.EBITDA;TR.EBIT;TR.NetIncomeBeforeTaxes;TR.NetIncome;TR.TtlCmnSharesOut',{'Period':'FY0','Scale':'6','curn':df.at[0,'Currency']})
dfFin2 = ld.get_data(RIC, 'TR.TotalRevenue;TR.GrossProfit;TR.EBITDA;TR.EBIT;TR.NetIncomeBeforeTaxes;TR.NetIncome;TR.TtlCmnSharesOut',{'Period':'FY-1','Scale':'6','curn':df.at[0,'Currency']})

# results as simple lists 
listFin = dfFin.iloc[0].tolist() #select row 0, all columns
listFin2 = dfFin2.iloc[0].tolist()

# results as single dictionary
dataDict2 = {'FY0': listFin, 'FY-1': listFin2 }

# Pandas DataFrame from dictionary 
dfIncome = pd.DataFrame(dataDict2, index = dfFin.columns) 
dfIncome        

,FY0,FY-1
Instrument,IBM.N,IBM.N
Total Revenue,62753,61860
Gross Profit,35552,34300
EBITDA,14435,13818
EBIT,9768,9422
Net Income Before Taxes,5797,8690
Net Income Incl Extra Before Distributions,6023,7502
Total Common Shares Outstanding,926.29007,915.01365


### Balance Sheet (MIL)

In [15]:
dfBal = ld.get_data(RIC, 'TR.CashAndSTInvestments;TR.PropertyPlantEquipmentTotalNet;TR.TotalAssetsReported;TR.CurrentLiabilities;TR.TotalLongTermDebt;TR.TotalLiabilities;TR.TotalEquity',{'Period':'FY0','Scale':6,'Curn':df.at[0,'Currency']})
dfBal2 = ld.get_data(RIC, 'TR.CashAndSTInvestments;TR.PropertyPlantEquipmentTotalNet;TR.TotalAssetsReported;TR.CurrentLiabilities;TR.TotalLongTermDebt;TR.TotalLiabilities;TR.TotalEquity',{'Period':'FY-1','Scale':6,'Curn':df.at[0,'Currency']})

# results as single dictionary
dataDict3 = {'FY0': list(dfBal.iloc[0, :]), 'FY-1': list(dfBal2.iloc[0, :]) }

# Pandas DataFrame from dictionary 
dfBalance= pd.DataFrame(dataDict3, index = dfBal.columns) 
dfBalance 

,FY0,FY-1
Instrument,IBM.N,IBM.N
Cash and Short Term Investments,14591,13441
"Property/Plant/Equipment, Total - Net",8928,8721
"Total Assets, Reported",137175,135241
Current Liabilities,33142,34122
Total Long Term Debt,49884,50121
Total Liabilities,109868,112708
Total Equity,27307,22533


### Cash Flow Statement (MIL)

In [17]:
dfCash = ld.get_data(RIC, 'TR.NetIncomeStartingLine',{'Period':'FY0','Scale':'6','curn':df.at[0,'Currency']})
dfCash_2 = ld.get_data(RIC, 'TR.NetIncomeStartingLine,TR.DepreciationDepletion,TR.Amortization,TR.NonCashItems,TR.DeferredTaxes',{'Period':'FY0','Scale':'6','Curn':df.at[0,'Currency']})
dfCash_3 = ld.get_data(RIC, 'TR.CashFromOperatingAct;TR.CapitalExpenditures;TR.CashFromInvestingAct;TR.CashDividendsPaid*-1;TR.CashFromFinancingAct',{'Period':'FY0','Scale':6,'Curn':df.at[0,'Currency']})
#insert as columns, entincome and  sum of cash from operations
dfCash_3['Net Income'] = [dfCash.at[0,'Net Income/Starting Line, Cumulative']]
dfCash_3['Cash From Operations'] = [sum(list(dfCash_2.iloc[0, 1:]))]

dfCash2 = ld.get_data(RIC, 'TR.NetIncomeStartingLine',{'Period':'FY-1','Scale':'6','curn':df.at[0,'Currency']})
dfCash2_2 = ld.get_data(RIC, 'TR.NetIncomeStartingLine,TR.DepreciationDepletion,TR.Amortization,TR.NonCashItems,TR.DeferredTaxes',{'Period':'FY-1','Scale':'6','Curn':df.at[0,'Currency']})
dfCash2_3 = ld.get_data(RIC, 'TR.CashFromOperatingAct;TR.CapitalExpenditures;TR.CashFromInvestingAct;TR.CashDividendsPaid*-1;TR.CashFromFinancingAct',{'Period':'FY-1','Scale':6,'Curn':df.at[0,'Currency']})
#insert as columns, entincome and  sum of cash from operations
dfCash2_3['Net Income'] = [dfCash2.at[0,'Net Income/Starting Line, Cumulative']]
dfCash2_3['Cash From Operations'] = [sum(list(dfCash2_2.iloc[0, 1:]))]

#DEBUG display(dfCash_3,dfCash2_3)

# results as simple lists
listCash = dfCash_3.iloc[0, :].tolist()
listCash2 = dfCash2_3.iloc[0, :].tolist()


# results as single dictionary
dataDict3 = {'FY0': listCash, 'FY-1': listCash2 }
#dataDict3

# Pandas DataFrame from dictionary 
dfCashFlow = pd.DataFrame(dataDict3, index = dfCash_3.columns) 
dfCashFlow  

,FY0,FY-1
Instrument,IBM.N,IBM.N
"Cash from Operating Activities, Cumulative",13446,13931
"Capital Expenditures, Cumulative",-1685,-1810
"Cash from Investing Activities, Cumulative",-4937,-7070
TR.CASHDIVIDENDSPAID*-1,-6147,-6040
"Cash from Financing Activities, Cumulative",-7079,-1769
Net Income,6023,7502
Cash From Operations,12132,11747


## Review of Stock Performance 

### Price Actions

In [18]:
# retrieve percentage changes
dfPriceActions = ld.get_data(RIC, 'TR.PricePctChg1M/100;TR.PricePctChg3M/100;TR.PricePctChg6M/100;TR.PricePctChg1Y/100')
dfPriceActions

#create lists out of results 
listNames = dfPriceActions.columns.tolist()
listPA = dfPriceActions.iloc[0, :].tolist()

# remove the first element, it's header
listNames.pop(0)
listPA.pop(0)
#display(listNames, listPA)

# Pandas DataFrame from dictionary
dataDictPA = {'Names': listNames, 'Periods': listPA } 
dfPA = pd.DataFrame(dataDictPA, index = listNames) 


import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyof
pyof.init_notebook_mode(connected=True)

fig = px.bar(dfPA, x='Names', y='Periods')
fig



### Profitability 

In [20]:
dfProfit = ld.get_data(RIC, 'TR.GrossMargin;TR.ResearchAndDevelopment;TR.TotalRevenue;TR.SGAExpenseTotal;TR.EBITDAMarginPercent;TR.OperProfitMarginPct;TR.PretaxMarginPercent;TR.NetProfitMargin;TR.CashFlowMarginPercent/100;TR.IncomeTaxRatePct;TR.ROATotalAssetsPercent;TR.ReturnonAvgTotEqtyPctNetIncomeBeforeExtraItems',{'Period':'FY0'})
#dfProfit   
dfProfit2 = ld.get_data(RIC, 
    'TR.GrossProfitMarginPct5YrAvg();TR.EBITDAMarginPct5YrAvg;TR.OperatingProfitMarginPct5YrAvg;TR.IncomeBefTaxMarginpct5YrAvg();TR.IncomeAftTaxMarginpct5YrAvg();TR.CashFlowMarginPct5YrAvg()/100;TR.IncomeTaxRatePct5YrAvg()/100;TR.ReturnonAvgTotAssetsPctIncomeAfterTax5YrAvg')
dfProfit3 = ld.get_data(RIC, 'TR.ResearchAndDevelopment(),TR.TotalRevenue(),TR.SGAExpenseTotal,TR.ReturnonAvgTotEqtyPctNetIncomeBeforeExtraItems',{'Period':'FY0','Sdate':0,'Edate':-4,'Frq':'FY'})
dfProfit3['RAD over TR'] = dfProfit3['Research And Development']/dfProfit3['Total Revenue']
dfProfit3['SGA over TR'] = dfProfit3['Selling/General/Administrative Expense, Total']/dfProfit3['Total Revenue']
#dfProfit2

#Create a dictionary out of results, and structure a single multi-row Data Frame
# initialise results per column 
dataDictProfit = {'FY0':
        [dfProfit.at[0,'Gross Margin, Percent'],  dfProfit.at[0,'Research And Development']/dfProfit.at[0,'Total Revenue'],
        dfProfit.at[0,'Selling/General/Administrative Expense, Total']/dfProfit.at[0,'Total Revenue'],
        dfProfit.at[0,'EBITDA Margin, Percent'],dfProfit.at[0,'Operating Profit Margin, Percent'],
        dfProfit.at[0,'Pretax Margin, Percent'],dfProfit.at[0,'Net Profit Margin, (%)'],
        dfProfit.at[0,'TR.CASHFLOWMARGINPERCENT/100'],dfProfit.at[0,'Income Tax Rate, %'],
        dfProfit.at[0,'ROA Total Assets, Percent'],dfProfit.at[0,'ROE Total Equity %']
    ]
        ,'5 Year Avg':[dfProfit2.at[0,'Gross Profit Margin, %, 5 Yr Avg'],dfProfit3['RAD over TR'].mean(),
                       dfProfit3['SGA over TR'].mean(), dfProfit2.at[0,'EBITDA Margin, %, 5 Yr Avg'],
                       dfProfit2.at[0,'Oper Profit Margin, %, 5 Yr Avg'],dfProfit2.at[0,'Income Bef Tax Margin, %, 5 Yr Avg'],
                       dfProfit2.at[0,'Income Aft Tax Margin, %, 5 Yr Avg'],dfProfit2.at[0,'TR.CASHFLOWMARGINPCT5YRAVG()/100'],
                       dfProfit2.at[0,'TR.INCOMETAXRATEPCT5YRAVG()/100'],dfProfit2.at[0,'ROA Tot Assets, %, 5 Yr Avg'],
                       dfProfit3['ROE Total Equity %'].mean()
]
                       
}
#dataDictProfit

# structure as required multi-row
dfMultiProfit = pd.DataFrame(dataDictProfit, index =[
    'Gross Margin','R/D % Sales','SGA % Sales','EBITDA Margin','Operating Margin','Pretax Margin','Net Margin',
    'CF/Sales','Tax Rate','ROA','ROE'
    ]) 
dfMultiProfit   

,FY0,5 Year Avg
Gross Margin,56.653870,55.393184
R/D % Sales,0.119182,0.112762
SGA % Sales,0.303507,0.314336
EBITDA Margin,23.002884,22.602059
Operating Margin,9.237805,7.744094
Pretax Margin,9.237810,7.744094
Net Margin,10.310260,8.199293
CF/Sales,0.177474,0.172620
Tax Rate,-11.609453,-0.058780
ROA,4.750088,3.511109


### FY0 Sales By Region 

In [21]:
dfSalesByRegion = ld.get_data(RIC, 'TR.BGS.GeoTotalRevenue.segmentName;TR.BGS.GeoTotalRevenue()',{'Scale':'6','curn':df.at[0,'Currency']})
dfSalesByRegion

,Instrument,Segment Name,Geographic Total Revenues (Calculated)
0,IBM.N,Americas,31266
1,IBM.N,Europe/Middle East/Africa (EMEA),19429
2,IBM.N,Asia/Pacific,12058
3,IBM.N,Segment Total,62753
4,IBM.N,Consolidated Total,62753


## Price Chart

### Pricing Data  - 1 -  Closing Prices

In [22]:

dfPricingIntermediate1 = ld.get_data(RIC, 'TR.PriceClose.calcdate;TR.PriceClose',{'sdate':'-5AY','edate':'0d','frq':'D'})
dfPricingIntermediate1


,Instrument,Calc Date,Price Close
0,IBM.N,2020-04-22,113.960617
1,IBM.N,2020-04-23,115.909151
2,IBM.N,2020-04-24,119.128054
3,IBM.N,2020-04-27,120.274251
4,IBM.N,2020-04-28,120.608558
...,...,...,...
1251,IBM.N,2025-04-14,239.06
1252,IBM.N,2025-04-15,240.7
1253,IBM.N,2025-04-16,238.57
1254,IBM.N,2025-04-17,238.81


### Pricing Data - 2 - Dates 

In [48]:
# Intermediate dates to use in the next step
import datetime
from datetime import datetime
from datetime import timedelta

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += timedelta(days=1)
    return days

RIC2 = 'MSFT.O'

dfDatesMSFTIntermediate1 = ld.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'0d'})
dfDatesMSFTIntermediate2 = ld.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'-24AM'})


# number of work days between the two dates + 250
daysBetween = len(workdays(dfDatesMSFTIntermediate2.at[0,'Date'],
               dfDatesMSFTIntermediate1.at[0,'Date'])) + 250
daysBetween
dfDatesMSFTIntermediate4 = ld.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'-'+str(daysBetween)+'D'})

display(str(dfDatesMSFTIntermediate1.at[0,'Date']),
str(dfDatesMSFTIntermediate2.at[0,'Date']),
str(dfPricingIntermediate1.at[0,'Calc Date']),
str(dfDatesMSFTIntermediate4.at[0,'Date']))

if dfPricingIntermediate1.at[0,'Calc Date'] > dfDatesMSFTIntermediate2.at[0,'Date']:
    laterDate1 = str(dfPricingIntermediate1.at[0,'Calc Date'])
else:
    laterDate1 = str(dfDatesMSFTIntermediate2.at[0,'Date'])
    

if dfPricingIntermediate1.at[0,'Calc Date'] > dfDatesMSFTIntermediate4.at[0,'Date']:
    laterDate2 = str(dfPricingIntermediate1.at[0,'Calc Date'])
else:
    laterDate2 = str(dfDatesMSFTIntermediate4.at[0,'Date'])
    
display(laterDate1,laterDate2)

'2025-04-21 00:00:00'

'2023-04-21 00:00:00'

'2020-04-22 00:00:00'

'2022-03-22 00:00:00'

'2023-04-21 00:00:00'

'2022-03-22 00:00:00'

### Pricing Data - 3 - Volumes, Prices, 50 days, 200 days 

In [49]:
dfDtVolPrice = ld.get_data(df_2.at[0,'Primary Quote RIC'], 'TR.PriceClose.CalcDate,TR.PriceClose',{'Sdate':'0D','Edate':laterDate2[:10],'Frq':'D'})

dfDtVolPrice2 = ld.get_data(df_2.at[0,'Primary Quote RIC'], 'TR.Volume',{'Sdate':'0D','Edate':laterDate2[:10],'Frq':'D','Scale':3})
dfDtVolPrice['Volume'] = dfDtVolPrice2['Volume']

#calculate moving averages
convolve50 = np.convolve(dfDtVolPrice['Price Close'], np.ones((50,))/50, mode='valid')
convolve200 = np.convolve(dfDtVolPrice['Price Close'], np.ones((200,))/200, mode='valid')
display(dfDtVolPrice,convolve50,convolve200)

,Instrument,Calc Date,Price Close,Volume
0,IBM.N,2025-04-21,236.22,1270.741
1,IBM.N,2025-04-17,238.81,1275.239
2,IBM.N,2025-04-16,238.57,1281.134
3,IBM.N,2025-04-15,240.7,820.351
4,IBM.N,2025-04-14,239.06,949.827
...,...,...,...,...
768,IBM.N,2022-03-28,131.47,767.957
769,IBM.N,2022-03-25,131.35,1000.289
770,IBM.N,2022-03-24,129.25,902.799
771,IBM.N,2022-03-23,128.3,1085.209


array([248.414 , 248.7584, 249.2482, 249.766 , 250.1666, 250.4994,
       250.9552, 250.9368, 250.7438, 250.8058, 250.7862, 250.7574,
       250.3528, 249.8384, 249.3274, 248.8074, 248.328 , 247.7588,
       247.1   , 246.497 , 245.9916, 245.5934, 245.1804, 244.5876,
       244.0474, 243.3846, 242.8226, 242.3622, 241.8674, 241.3766,
       240.6772, 239.9136, 239.4182, 238.7946, 238.3098, 237.8926,
       237.4602, 237.0408, 236.5264, 236.0058, 235.3684, 234.8996,
       234.2998, 233.6832, 233.0018, 232.324 , 231.6884, 231.1106,
       230.5932, 230.1304, 229.543 , 228.9222, 227.9482, 226.864 ,
       225.8112, 224.797 , 223.8114, 223.4572, 223.1612, 222.95  ,
       222.7284, 222.4814, 222.2882, 221.9544, 221.585 , 221.2968,
       221.0306, 220.7736, 220.6342, 220.4974, 220.3272, 220.2158,
       220.4174, 220.6094, 220.8456, 221.093 , 221.3456, 221.5634,
       221.7248, 221.9418, 222.1684, 222.3616, 222.5692, 222.7382,
       222.7012, 222.6346, 222.4726, 222.222 , 222.0066, 221.7

array([222.1895 , 221.8949 , 221.57635, 221.24825, 220.899  , 220.56305,
       220.24865, 219.97595, 219.6617 , 219.42615, 219.15   , 218.8601 ,
       218.4887 , 218.0844 , 217.6777 , 217.281  , 216.9129 , 216.5319 ,
       216.1212 , 215.7086 , 215.2954 , 214.90245, 214.5201 , 214.0868 ,
       213.6873 , 213.27075, 212.88345, 212.5078 , 212.1281 , 211.7507 ,
       211.3158 , 210.85325, 210.45465, 210.0392 , 209.60995, 209.1968 ,
       208.77035, 208.33555, 207.90585, 207.459  , 206.9927 , 206.51385,
       206.0136 , 205.51415, 205.0298 , 204.56055, 204.10025, 203.66575,
       203.31275, 202.97735, 202.62515, 202.26585, 201.8567 , 201.4499 ,
       201.065  , 200.69275, 200.31275, 200.0991 , 199.901  , 199.7269 ,
       199.552  , 199.3675 , 199.1909 , 199.0241 , 198.84455, 198.6804 ,
       198.53505, 198.4003 , 198.2558 , 198.101  , 197.9393 , 197.779  ,
       197.63545, 197.4889 , 197.34765, 197.20385, 197.06975, 196.93935,
       196.8038 , 196.6404 , 196.5105 , 196.3764 , 

### Price Chart 

In [50]:
from plotly.subplots import make_subplots

#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
#       x=dfDtVolPrice['Calc Date'],  #we can optionally chart the complete set
        x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],    # we are cutting off
        y=dfDtVolPrice['Price Close'],
        name = 'Price Close'), 
    secondary_y=False,
)
fig.add_trace(go.Scatter(
#    x=dfDtVolPrice['Calc Date'],
    x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],
    y=dfDtVolPrice['Volume'],
    name = 'Volume'), 
    secondary_y=True,
)
fig.add_trace(go.Scatter(
#    x=dfDtVolPrice['Calc Date'],
    x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],
    y=convolve50,
    name='50 Day MA',),
    secondary_y=False,
)
fig.add_trace(go.Scatter(
    x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],
#    x=dfDtVolPrice['Calc Date'],
    y=convolve200,
    name='200 Day MA',),
    secondary_y=False,
)

fig.update_yaxes(title_text="Price", secondary_y=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)

fig

In [51]:
dfPriceActions = ld.get_data(RIC, 'TR.PricePctChg1M/100;TR.PricePctChg3M/100;TR.PricePctChg6M/100;TR.PricePctChg1Y/100')
dfPriceActions

,Instrument,TR.PRICEPCTCHG1M/100,TR.PRICEPCTCHG3M/100,TR.PRICEPCTCHG6M/100,TR.PRICEPCTCHG1Y/100
0,IBM.N,-0.031369,0.053331,0.019288,0.300914


In [52]:
dfPricingIntermediate1 = ld.get_data(RIC, 'TR.PriceClose.calcdate;TR.PriceClose',{'sdate':'-5AY','edate':'0d','frq':'D'})
dfPricingIntermediate1

,Instrument,Calc Date,Price Close
0,IBM.N,2020-04-22,113.960617
1,IBM.N,2020-04-23,115.909151
2,IBM.N,2020-04-24,119.128054
3,IBM.N,2020-04-27,120.274251
4,IBM.N,2020-04-28,120.608558
...,...,...,...
1251,IBM.N,2025-04-14,239.06
1252,IBM.N,2025-04-15,240.7
1253,IBM.N,2025-04-16,238.57
1254,IBM.N,2025-04-17,238.81
